<a href="https://colab.research.google.com/github/agusdwiandika07/Pertemuan4dataset/blob/main/Uts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [3]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

/content/gdrive/My Drive/Kaggle


In [7]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
import math
import seaborn as sns
from statistics import mean
from math import sqrt

In [9]:
ls

covid_impact_on_airport_traffic.csv  kaggle.json  letter-recognition.csv


In [11]:
data = pd.read_csv("letter-recognition.csv",header=0)
data.head(10) 

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
5,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
6,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
7,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
8,J,2,2,4,4,2,10,6,2,6,12,4,8,1,6,1,7
9,M,11,15,13,9,7,13,2,6,2,12,1,9,8,1,1,8


In [13]:
##########################################################
# 3. SHARE TO TEST AND TRAIN DATA
##########################################################
x = data.iloc[:, 1:]
y = data['letter'].tolist()
print(x)

# Select 4000 rows data as a testing dataset
x_test = x.iloc[0:4000, :].values.astype('float32') # all pixel values 
y_test = y[0:4000] # Select label for testing data
x_train = x.iloc[4000:, :].values.astype('float32') # all pixel values 
y_train = y[4000:]

# # Share test and train data
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

       xbox   ybox   width   height  onpix   xbar   ybar   x2bar  y2bar   \
0          2      8       3       5       1      8     13      0       6   
1          5     12       3       7       2     10      5      5       4   
2          4     11       6       8       6     10      6      2       6   
3          7     11       6       6       3      5      9      4       6   
4          2      1       3       1       1      8      6      6       6   
...      ...    ...     ...     ...     ...    ...    ...    ...     ...   
19995      2      2       3       3       2      7      7      7       6   
19996      7     10       8       8       4      4      8      6       9   
19997      6      9       6       7       5      6     11      3       7   
19998      2      3       4       2       1      8      7      2       6   
19999      4      9       6       6       2      9      5      3       1   

       xybar   x2ybar  xy2bar  xedge   xedgey  yedge   yedgex  
0           6      10  

In [14]:
##########################################################
# 5. FEATURE NORMALIZATION FOR BOTH (TEST & TRAIN)
##########################################################
# Proceed to normalize the features because the pixel intensities are currently between the range of 0 and 255
print((min(x_train[2]), max(x_train[2])))

# Normalizing the data
x_train = x_train/255.0
x_test = x_test/255.0

# Printing the shape of the Datasets
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(1.0, 12.0)
x_train shape: (16000, 16)
16000 train samples
4000 test samples


In [15]:
##########################################################
# 6. TRAIN SVM ALGORITHM
##########################################################

model_clf = svm.SVC(C=2000, kernel='linear',gamma=0.0001)

# Train the svm algorithm
model_clf.fit(x_train, y_train)

SVC(C=2000, gamma=0.0001, kernel='linear')

In [17]:
##########################################################
# 6. APPLY THE TRAINED LEARNER TO TEST NEW DATA
##########################################################
# Apply the trained svm to make prediction of test data
y_pred = model_clf.predict(x_test)

In [18]:
##########################################################
# 7. MULTI-CLASS CONFUSION MATRIX FOR EACH CLASS
##########################################################

# Actual and predicted classes
lst_actual_class = y_test
lst_predicted_class = y_pred

# Class = Label A-Z
lst_classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]


# Compute multi-class confusion matrix
arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)
# Temp store results
store_sens = [];
store_spec = [];
store_acc = [];
store_bal_acc = [];
store_prec = [];
store_fscore = [];
store_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]
    
    
    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);
    
    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    store_sens.append(sensitivity);
    store_spec.append(specificity);
    store_acc.append(accuracy);
    store_bal_acc.append(balanced_accuracy);
    store_prec.append(precision);
    store_fscore.append(f1Score);
    store_mcc.append(MCC);
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Sensitivity: {0}".format(sensitivity));
    print("Specificity: {0}".format(specificity));
    print("Accuracy: {0}".format(accuracy));
    print("Balanced Accuracy: {0}".format(balanced_accuracy));
    
    print("Precision: {0}".format(precision));
    print("F1-Score: {0}".format(f1Score));
    print("MCC: {0}\n".format(MCC));

Print Class: 0
TP=152, FP=30, TN=3810, FN=8
Sensitivity: 0.95
Specificity: 0.992
Accuracy: 0.99
Balanced Accuracy: 0.971
Precision: 0.835
F1-Score: 0.889
MCC: 0.886

Print Class: 1
TP=141, FP=39, TN=3801, FN=19
Sensitivity: 0.881
Specificity: 0.99
Accuracy: 0.986
Balanced Accuracy: 0.936
Precision: 0.783
F1-Score: 0.829
MCC: 0.823

Print Class: 2
TP=137, FP=13, TN=3820, FN=30
Sensitivity: 0.82
Specificity: 0.997
Accuracy: 0.989
Balanced Accuracy: 0.908
Precision: 0.913
F1-Score: 0.864
MCC: 0.86

Print Class: 3
TP=147, FP=37, TN=3797, FN=19
Sensitivity: 0.886
Specificity: 0.99
Accuracy: 0.986
Balanced Accuracy: 0.938
Precision: 0.799
F1-Score: 0.84
MCC: 0.834

Print Class: 4
TP=124, FP=24, TN=3825, FN=27
Sensitivity: 0.821
Specificity: 0.994
Accuracy: 0.987
Balanced Accuracy: 0.908
Precision: 0.838
F1-Score: 0.829
MCC: 0.823

Print Class: 5
TP=129, FP=26, TN=3828, FN=17
Sensitivity: 0.884
Specificity: 0.993
Accuracy: 0.989
Balanced Accuracy: 0.938
Precision: 0.832
F1-Score: 0.857
MCC: 0

In [19]:
##########################################################
# 8. OVERALL - FINAL PREDICTION PERFORMANCE
##########################################################

print("Overall Performance Prediction:");
print("Sensitivity: {0}%".format(round(mean(store_sens)*100, 4)));
print("Specificity: {0}%".format(round(mean(store_spec)*100, 4)));
print("Accuracy: {0}%".format(round(mean(store_acc)*100, 4)));
print("Balanced Accuracy: {0}%".format(round(mean(store_bal_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(store_prec)*100, 4)));
print("F1-Score: {0}%".format(round(mean(store_fscore)*100, 4)))
print("MCC: {0}\n".format(round(mean(store_mcc), 4)))

Overall Performance Prediction:
Sensitivity: 85.0846%
Specificity: 99.4077%
Accuracy: 98.8577%
Balanced Accuracy: 92.2423%
Precision: 85.3654%
F1-Score: 85.0923%
MCC: 0.8457

